In [1]:
import json
import sys
import os
import torch
dirs = ["../..", "../create_prompt_llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import prompt_rag

search_engine = "brave"


with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)
try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)

def save_diff(x, model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
        json.dump(x, f, indent=4)

def load_diff(model_name, diff_version):
    try:
        with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
            return json.load(f)
    except:
        data_init = [{"claim": i["claim"], "label": int(i["label"])} for i in data_search_llm]
        with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
            json.dump(data_init, f, indent=4)
        return data_init


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256'

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
# model_type = ModelType.llama3_70b_instruct_int4

llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    # max_model_len=4096,
    # gpu_memory_utilization=0.95,
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 32,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True,
)


2024-05-10 12:33:58,973 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-05-10 12:33:58,975 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-05-10 12:33:59,001 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 2a8987246d4b67f321effea600434525 and a total number of 976 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "ma

WARNING 05-10 12:33:59 config.py:1086] Casting torch.bfloat16 to torch.float16.
INFO 05-10 12:33:59 config.py:177] The model is convertible to Marlin format. Using Marlin kernel.


2024-05-10 12:34:02,736	INFO worker.py:1749 -- Started a local Ray instance.


INFO 05-10 12:34:03 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', speculative_config=None, tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42, served_model_name=/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4)
INFO 05-10 12:34:08 utils.py:660] Found nccl from library /home/hanlv/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=2574827) INFO 05-10 12:34:08 utils.py:660] Found nccl from library /home/hanlv/.config/vllm/nccl/cu

先用llama3根据检索到的内容给出每个claim的结论


In [10]:
def get_prompt_for_diff_conclusions(
        claim, claim_date, search_engine, search_results,
        model_name="llama3", K=5, ids=None,
):
    """
    为了验证先验知识和label的差异性，先对检索到的信息进行总结
    """
    claim = claim.strip()
    if model_name == "llama3":
        pre = "Below is a claim and some information searched online. These pieces of Information are relevant to the claim. This claim and all information include their respective publication dates and contents. To classify the claim more accurately (if the content described by the claim is correct, it will be classified as true; if the content described by the claim is incorrect, it will be classified as false), please judge the correctness of the claim based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the claim and the date of publication of the information. Just provide the final conclusion without explaining the reasoning process. \n\n"
    elif model_name == "mixtral":
        pre = "Below is a claim and some information searched online. These pieces of Information are relevant to the claim. This claim and all information include their respective publication dates and contents. To classify the claim more accurately (if the content described by the claim is correct, it will be classified as true; if the content described by the claim is incorrect, it will be classified as false), please judge the correctness of the claim based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the claim and the date of publication of the information. Just provide the brief final conclusion without explaining the reasoning process. \n\n"

    else:
        raise Exception("Select model name in [\"llama3\"].")
    
    if search_engine == "brave":
        if ids is None:
            ids = slice(0, K)
        snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
    else:
        raise Exception("Select search engine in [\"brave\"].")
    
    text = "Claim:" + prompt_rag.get_claim_with_date(claim, claim_date) +'\n\n'
    info = "Information:\n" + snippet
    return pre + text + info

def update_conclusions(
        model_name, diff_version, get_resp_list, search_engine, data_search, K=5):
    
    data_diff = load_diff(model_name, diff_version)

    len_data_diff = 0
    for i, item in enumerate(data_diff):
        if item.get("conclusion") is None:
            len_data_diff = i
            break

    prompt_list = []
    for i in range(len_data_diff, len(data_diff)):
        item = data_search[i]
        ids = None
        prompt = get_prompt_for_diff_conclusions(
            item["claim"], item["date"], 
            search_engine, item[f"{search_engine}_search_results"], 
            model_name, K=K, ids=ids
        )
        prompt_list.append(prompt)
    request_list = [{'query': prompt} for prompt in prompt_list]
    resp_list = get_resp_list(request_list)
    resp_list = [i["response"] for i in resp_list]

    for i in range(len_data_diff, len(data_diff)):
        data_diff[i][f"conclusion"] = resp_list[i - len_data_diff].strip()

    save_diff(data_diff, model_name, diff_version)


In [4]:
model_name = "llama3" # llama3
diff_version = 1

# update_conclusions(model_name, diff_version, get_resp_list, search_engine, data_search)


In [13]:
def update_diff(model_name, diff_version, get_resp_list):

    data_diff = load_diff(model_name, diff_version)

    len_data_diff = 0
    for i, item in enumerate(data_diff):
        if item.get("match") is None:
            len_data_diff = i
            break

    prompt_list = []
    for i in range(len_data_diff, len(data_diff)):

        item = data_diff[i]
        claim = item["claim"]
        label = int(item["label"])
        if not isinstance(label, int):
            raise Exception(f"Label 不是 int 类型。\n{claim}")
        
        if label == 1:
            prompt_list.append("Do not output.")
            continue

        answer = item["conclusion"]

        if label == 2:
            ground_truth = f'The claim({claim}) is true.'
        elif label == 0:
            ground_truth = f'The claim({claim}) is false.'
        else:
            # label为1不做计算，ground_truth是什么无所谓
            pass
        
        if model_name == "llama3":
            prompt = f"Does the answer given below match the ground truth? If it does, please answer \"Yes\"; otherwise, answer \"No\".\nGround truth: {ground_truth}\nAnswer:\n{answer}"
        elif model_name == "mixtral":
            prompt = f"Does the answer given below match the ground truth? If it does, please only answer \"Yes\"; otherwise, only answer \"No\". And don't explain the reason. \nGround truth: {ground_truth}\nAnswer:\n{answer}"
        else:
            raise Exception()
        prompt_list.append(prompt)
    request_list = [{'query': prompt} for prompt in prompt_list]
    resp_list = get_resp_list(request_list)
    resp_list = [i["response"] for i in resp_list]

    for i in range(len_data_diff, len(data_diff)):
        item = data_diff[i]
        claim = item["claim"]
        label = int(item["label"])
        if not isinstance(label, int):
            raise Exception(f"Label 不是 int 类型。\n{claim}")
        
        if label == 1:
            data_diff[i][f"match"] = 'None'
        else:
            data_diff[i][f"match"] = resp_list[i - len_data_diff].strip()

    save_diff(data_diff, model_name, diff_version)


In [5]:
model_name = "llama3" # llama3
diff_version = 1

# update_diff(model_name, diff_version, get_resp_list)


手动判定match

In [10]:
match_v2 = "match2"
model_name = "llama3" # llama3
diff_version = 1
data_diff = load_diff(model_name, diff_version)

cnt_false = 0
cnt_true = 0

for i in range(len(data_diff)):
    item = data_diff[i]
    claim = item["claim"]
    label = int(item["label"])
    if not isinstance(label, int):
        raise Exception(f"Label 不是 int 类型。\n{claim}")
    # print(item)
    if "true" in item["conclusion"].lower():
        pred = 2
    elif "false" in item["conclusion"].lower() or "unverified" in item["conclusion"].lower()\
        or "unverifiable" in item["conclusion"].lower():
        pred = 0
    else:
        pred = 0
    if label == 1:
        data_diff[i][match_v2] = 'None'
    else:
        if pred == label:
            data_diff[i][match_v2] = "Yes"
        else:
            data_diff[i][match_v2] = "No"
cnt_true, cnt_false
# save_diff(data_diff, model_name, diff_version)


In [9]:
# 检查match和match2冲突的部分
# 最终生成match_final

data_diff = load_diff(model_name, diff_version)
cnt = 0
for i in range(len(data_diff)):
    item = data_diff[i]
    claim = item["claim"]
    label = int(item["label"])
    if  item["match2"] != item["match"]:
        if label == 0 and item['conclusion'] == 'TRUE' and item["match2"] == 'No':
            continue
        item.pop("claim")
        print(item)
cnt


{'label': 0, 'conclusion': 'Based on the available information, I classify the claim as UNVERIFIED.', 'match': 'No', 'match2': 'Yes'}
{'label': 2, 'conclusion': 'TRUE', 'match': 'No', 'match2': 'Yes'}
{'label': 0, 'conclusion': 'Based on the available information, I conclude that the claim is UNVERIFIED.', 'match': 'No', 'match2': 'Yes'}
{'label': 0, 'conclusion': 'Based on the available information, I conclude that the claim is FALSE.', 'match': 'No', 'match2': 'Yes'}
{'label': 0, 'conclusion': 'Based on the available information, I conclude that the claim is PARTLY FALSE.', 'match': 'No', 'match2': 'Yes'}
{'label': 0, 'conclusion': 'Based on the available information, I conclude that the claim is FALSE.', 'match': 'No', 'match2': 'Yes'}
{'label': 0, 'conclusion': 'Based on the available information, I conclude that the claim is FALSE.', 'match': 'No', 'match2': 'Yes'}
{'label': 0, 'conclusion': 'Based on the available information, I conclude that the claim "WHO confirmed COVID-19 as 

0